# Bayesian Logistic Regression (MCMC: NUTS Sampler)

Chapter 10 highlights the No-U-Turn Sampler (NUTS), an adaptive variant of
Hamiltonian Monte Carlo (HMC).

This notebook implements a compact NUTS sampler (identity mass matrix) for
Bayesian logistic regression on a **small** training set, and then produces
out-of-sample predictions and runs the shared backtest.

Note: NUTS is computationally expensive; we intentionally limit the universe
and training samples to keep runtime reasonable.


# Chapter 10 Bayesian ML (Predictive): Bayesian Logistic Regression (MCMC: NUTS)

These notebooks mirror the *methods* highlighted in
`ml_finance_thoery/machine-learning-for-trading/10_bayesian_machine_learning/README.md`
and apply them to the local `dataset/cleaned/` asset universe to produce
out-of-sample predictions and a backtest using the same **vectorized** engine
used by the `notebooks/ML_Linear_Models_*` notebooks.


In [1]:
from __future__ import annotations

import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'

# Ensure `src/` is on sys.path so `backtester` is importable
src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [2]:
from scipy.special import expit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

ASSET_LIMIT = 15
N_TRAIN_SAMPLES = 6_000
WARMUP = 300
SAMPLES = 300
MAX_DEPTH = 8
TARGET_ACCEPT = 0.65
TAU2 = 10.0**2

def compute_features_fast(ohlcv: pd.DataFrame) -> pd.DataFrame:
    df = ohlcv.copy().sort_index()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    ret_1d = c.pct_change()
    hl_range_pct = (h - l) / c.replace(0.0, np.nan)
    vol_z_20 = (v - v.rolling(20).mean()) / v.rolling(20).std()
    out = pd.DataFrame({'ret_1d': ret_1d, 'hl_range_pct': hl_range_pct, 'vol_z_20': vol_z_20}).sort_index()
    out['ret_1d_fwd'] = out['ret_1d'].shift(-1)
    out['y_up_fwd'] = (out['ret_1d_fwd'] > 0).astype(int)
    return out

from backtester.data import load_cleaned_assets
files = sorted([p for p in CLEANED_DIR.glob('Asset_*.csv')])
symbols = [p.stem for p in files][:ASSET_LIMIT]
assets_ohlcv = load_cleaned_assets(symbols=symbols, cleaned_dir=str(CLEANED_DIR))

frames = []
for sym, df in assets_ohlcv.items():
    feat = compute_features_fast(df)
    feat['Asset_ID'] = sym
    frames.append(feat)
data = pd.concat(frames, axis=0).sort_index().dropna(subset=['y_up_fwd'])

# Time split
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(data.index.unique()).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))
val_start = align_to_trading_date(idx, pd.Timestamp(raw_val_start))
train_start = align_to_trading_date(idx, pd.Timestamp(raw_train_start))

df_train = data.loc[(data.index >= train_start) & (data.index < val_start)].copy()
df_test = data.loc[(data.index >= test_start) & (data.index <= end)].copy()

feature_cols = [c for c in df_train.columns if c not in {'Asset_ID', 'ret_1d_fwd', 'y_up_fwd'}]
X_train = df_train[feature_cols].replace([np.inf, -np.inf], np.nan)
y_train = df_train['y_up_fwd'].astype(int).to_numpy()
X_test = df_test[feature_cols].replace([np.inf, -np.inf], np.nan)

imp = SimpleImputer(strategy='median')
scaler = StandardScaler()
Xtr = scaler.fit_transform(imp.fit_transform(X_train))
Xte = scaler.transform(imp.transform(X_test))
Xtr = np.hstack([np.ones((Xtr.shape[0], 1)), Xtr])
Xte = np.hstack([np.ones((Xte.shape[0], 1)), Xte])

# Subsample for NUTS
if Xtr.shape[0] > N_TRAIN_SAMPLES:
    idx_s = rng.choice(Xtr.shape[0], size=N_TRAIN_SAMPLES, replace=False)
    Xtr = Xtr[idx_s]
    y_train = y_train[idx_s]

d = Xtr.shape[1]

def logp_and_grad(theta: np.ndarray) -> tuple[float, np.ndarray]:
    # log posterior up to constant
    z = Xtr @ theta
    p = expit(z)
    eps = 1e-12
    ll = float(np.sum(y_train * np.log(p + eps) + (1 - y_train) * np.log(1 - p + eps)))
    lp = float(-0.5 * np.sum(theta[1:] ** 2) / TAU2)
    # gradient
    g = Xtr.T @ (y_train - p)
    g[1:] += -(theta[1:] / TAU2)
    return ll + lp, g

def leapfrog(theta: np.ndarray, r: np.ndarray, grad: np.ndarray, eps: float) -> tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    r_half = r + 0.5 * eps * grad
    theta_new = theta + eps * r_half
    lp_new, grad_new = logp_and_grad(theta_new)
    r_new = r_half + 0.5 * eps * grad_new
    return theta_new, r_new, grad_new, lp_new

def stop_criterion(theta_minus: np.ndarray, theta_plus: np.ndarray, r_minus: np.ndarray, r_plus: np.ndarray) -> bool:
    dtheta = theta_plus - theta_minus
    return (np.dot(dtheta, r_minus) >= 0.0) and (np.dot(dtheta, r_plus) >= 0.0)

def build_tree(theta: np.ndarray, r: np.ndarray, grad: np.ndarray, log_u: float, v: int, j: int, eps: float, theta0: np.ndarray, r0: np.ndarray, lp0: float) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, int, bool, float, int]:
    if j == 0:
        theta1, r1, grad1, lp1 = leapfrog(theta, r, grad, v * eps)
        joint = lp1 - 0.5 * np.dot(r1, r1)
        n1 = 1 if log_u <= joint else 0
        s1 = (log_u < (joint + 1000.0))
        alpha = min(1.0, math.exp(joint - (lp0 - 0.5 * np.dot(r0, r0))))
        return theta1, r1, theta1, r1, theta1, n1, s1, alpha, 1
    theta_minus, r_minus, theta_plus, r_plus, theta1, n1, s1, alpha1, n_alpha1 = build_tree(theta, r, grad, log_u, v, j - 1, eps, theta0, r0, lp0)
    if s1:
        if v == -1:
            theta_minus, r_minus, _, _, theta2, n2, s2, alpha2, n_alpha2 = build_tree(theta_minus, r_minus, logp_and_grad(theta_minus)[1], log_u, v, j - 1, eps, theta0, r0, lp0)
        else:
            _, _, theta_plus, r_plus, theta2, n2, s2, alpha2, n_alpha2 = build_tree(theta_plus, r_plus, logp_and_grad(theta_plus)[1], log_u, v, j - 1, eps, theta0, r0, lp0)
        if (n1 + n2) > 0 and rng.random() < (n2 / (n1 + n2)):
            theta1 = theta2
        s1 = s2 and stop_criterion(theta_minus, theta_plus, r_minus, r_plus)
        alpha1 += alpha2
        n_alpha1 += n_alpha2
        n1 += n2
    return theta_minus, r_minus, theta_plus, r_plus, theta1, n1, s1, alpha1, n_alpha1

# Dual averaging for step size during warmup
eps = 0.1
mu_da = math.log(10.0 * eps)
hbar = 0.0
eps_bar = 1.0
gamma = 0.05
t0 = 10.0
kappa = 0.75

theta = np.zeros(d)
lp, grad = logp_and_grad(theta)
draws = []
for m in range(1, WARMUP + SAMPLES + 1):
    r0 = rng.normal(size=d)
    joint0 = lp - 0.5 * np.dot(r0, r0)
    log_u = joint0 + math.log(rng.random())
    theta_minus = theta.copy(); theta_plus = theta.copy()
    r_minus = r0.copy(); r_plus = r0.copy()
    theta_prop = theta.copy()
    n = 1
    s = True
    alpha = 0.0
    n_alpha = 0
    for j in range(MAX_DEPTH):
        v = -1 if rng.random() < 0.5 else 1
        if v == -1:
            theta_minus, r_minus, _, _, theta_prime, n_prime, s_prime, alpha_prime, n_alpha_prime = build_tree(theta_minus, r_minus, logp_and_grad(theta_minus)[1], log_u, v, j, eps, theta, r0, lp)
        else:
            _, _, theta_plus, r_plus, theta_prime, n_prime, s_prime, alpha_prime, n_alpha_prime = build_tree(theta_plus, r_plus, logp_and_grad(theta_plus)[1], log_u, v, j, eps, theta, r0, lp)
        if not s_prime:
            break
        if rng.random() < (n_prime / n):
            theta_prop = theta_prime
        n += n_prime
        s = s_prime and stop_criterion(theta_minus, theta_plus, r_minus, r_plus)
        alpha += alpha_prime
        n_alpha += n_alpha_prime
        if not s:
            break
    theta = theta_prop
    lp, grad = logp_and_grad(theta)
    accept_rate = alpha / max(1, n_alpha)
    if m <= WARMUP:
        hbar = (1 - 1 / (m + t0)) * hbar + (1 / (m + t0)) * (TARGET_ACCEPT - accept_rate)
        log_eps = mu_da - (math.sqrt(m) / gamma) * hbar
        eps = math.exp(log_eps)
        eta = m ** (-kappa)
        eps_bar = math.exp((1 - eta) * math.log(eps_bar) + eta * log_eps)
        if m % 100 == 0:
            print('warmup', m, 'eps', eps, 'accept', accept_rate)
    else:
        draws.append(theta.copy())

B = np.stack(draws, axis=0)
print('posterior draws:', B.shape)

# Posterior predictive
p_samps = expit(Xte @ B.T)
p_mean = p_samps.mean(axis=1)
pred_long = pd.DataFrame({'Date': df_test.index, 'Asset_ID': df_test['Asset_ID'].to_numpy(), 'signal': p_mean - 0.5})
pred_matrix = pred_long.pivot_table(index='Date', columns='Asset_ID', values='signal', aggfunc='mean').sort_index()


warmup 100 eps 0.03891690411472329 accept 1.0
warmup 200 eps 0.024856216159904056 accept 0.2984713835093902
warmup 300 eps 0.024878289040988297 accept 0.4085464179059279
posterior draws: (300, 4)


In [3]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

# IMPORTANT: performance metrics should start when the model has signals.
# Keep the original prediction date range before reindexing to market data.
pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

# Align prediction matrix to available trade dates
pred_matrix = pred_matrix.reindex(close_prices.index)
# Restrict backtest to the prediction window (avoid 2016-start metrics).
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Bayes Logistic Regression (NUTS)'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)


Loading BokehJS ...

,style,Total Return [%],CAGR [%],Sharpe,Max Drawdown [%]
0,1N,40.482428,25.433826,1.269686,-19.680988
1,MPT,22.076325,14.223194,0.800078,-25.489476


,Bayes Logistic Regression (NUTS) - 1N
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1404824.280434
Equity Peak,1424496.884119
Total Return [%],40.482428
CAGR [%],25.433826
Volatility (ann) [%],19.252787
Sharpe,1.269686


,Bayes Logistic Regression (NUTS) - MPT
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1220763.253871
Equity Peak,1232089.533696
Total Return [%],22.076325
CAGR [%],14.223194
Volatility (ann) [%],18.814633
Sharpe,0.800078
